<a href="https://colab.research.google.com/github/tuomaseerola/music_and_science_seminar/blob/master/ATMS_seminar1_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Music and Science – Tutorial on Audio Analysis

[Tuomas Eerola](https://www.durham.ac.uk/staff/tuomas-eerola/), Durham University, Music Department, 2023.

For **Advanced Studies in Music and Science** Module

_Last update 21/2/2023 by TE_

# Section 1: Preliminaries

Welcome to Colab. This is a notebook that can run python code within a cloud service and offer the results within the same web page. You can edit any code blocks and try out what happens. Note that your changes will not have permanent impact on the notebook, as this notebook will be reset when you leave the page.

We are going to go through the basics of analysing audio examples. There are six _Learning Tasks_ to prompt you to try out the analysis.

To start the analysis, click **play button** on the top left corner of the code below. This will setup the system for this notebook. [Librosa](https://librosa.org/doc/0.8.1/) is the audio analysis tool (created by Brian McFee and others, see their [paper](https://conference.scipy.org/proceedings/scipy2015/pdfs/brian_mcfee.pdf)) that we will be using and the rest of the libraries that are activated here are for convenience and creating graphs.

In [ ]:
import numpy as np
import librosa
import librosa.display
import IPython.display as ipd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
print(librosa.__version__)

# Section 2: Read an audio file

## Section 2.1: Read Librosa example excerpts

Here are some example excerpts that come with Librosa. We can use any of them in the subsequent sections. Just remove the hashtag (#) in front of the line to get that audio file.

The following code also shows how to select only a _segment_ of an audio file. This is done by keywords `offset` and `duration`. Offset specifies where you want to start the segment (in seconds) and duration (also in seconds) is self-explanatory.

In [ ]:
#filename = librosa.ex('trumpet')   # Mihai Sorohan - Monophonic trumpet recording
#filename = librosa.ex('brahms')    # Hungarian Dance no. 5
#filename = librosa.ex('choice')    # A short drum and bass loop
filename = librosa.ex('fishin')     # Karissa Hobbs - Let’s Go Fishin’ A folk/pop song with verse/chorus/verse structure and vocals
#filename = librosa.ex('nutcracker')# Tchaikovsky - Dance of the Sugar Plum Fairy
#filename = librosa.ex('vibeace')   # 60-second clip
x, sr = librosa.load(filename, duration=20) # if you want to start the segment from a specific point, add offset = [2 for example] in the command

plt.figure(figsize=(12, 4))                  # create figure
librosa.display.waveshow(y = x,sr = sr)      # plot waveform
ipd.display(ipd.Audio(data = x, rate = sr))  # create playback object

## Section 2.2: Load audio files from online sources
This is how to load example sound files from Github pages (or anywhere from online, if you know the URL). These four instrument sounds have been taken from Vienna Symphonic Library (VSL) and uploaded to my Github pages. 

In [ ]:
import soundfile as sf
import io
from six.moves.urllib.request import urlopen

url1 = "https://raw.githubusercontent.com/tuomaseerola/audio/master/clar_one_note.wav"
url2 = "https://raw.githubusercontent.com/tuomaseerola/audio/master/trumpet_one_note.wav"
url3 = "https://raw.githubusercontent.com/tuomaseerola/audio/master/harp_one_note.wav"
url4 = "https://raw.githubusercontent.com/tuomaseerola/audio/master/harpsichord_one_note.wav"

clarinet, sr = sf.read(io.BytesIO(urlopen(url1).read()))
trumpet, sr = sf.read(io.BytesIO(urlopen(url2).read()))
harp, sr = sf.read(io.BytesIO(urlopen(url3).read()))
harpsichord, sr = sf.read(io.BytesIO(urlopen(url4).read()))

fig,axs = plt.subplots(2,2, sharex=True, figsize=(10, 6))
plt.figure(figsize=(12, 4))
librosa.display.waveplot(y = clarinet,sr = sr, max_sr = 100, color = 'purple',ax=axs[0,0]);axs[0, 0].set_title('Clarinet')
librosa.display.waveplot(y = trumpet,sr = sr, max_sr = 100, color = 'red',ax=axs[0,1]);axs[0, 1].set_title('Trumpet')
librosa.display.waveplot(y = harp,sr = sr, max_sr = 100, color = 'green',ax=axs[1,0]);axs[1, 0].set_title('Harp')
librosa.display.waveplot(y = harpsichord,sr = sr, max_sr = 100, color = 'blue',ax=axs[1,1]);axs[1, 1].set_title('Harpsichord')
ipd.display(ipd.Audio(data = clarinet, rate = sr))
ipd.display(ipd.Audio(data = trumpet, rate = sr))
ipd.display(ipd.Audio(data = harp, rate = sr))
ipd.display(ipd.Audio(data = harpsichord, rate = sr))


---
### _Learning Task 1_
---
_Try out some of the audio examples yourself and be sure you understand the difference between `URL` (address), and `clarinet/trumpet...` (this is where the signal is) and `sr` (the last stands for _sampling rate_)_. The next tasks will focus on extracting properties from the specific files, so understanding this is important.

Finally, your own code **WILL NOT BE SAVED** within the notebook so when you create the solutions to Learning Tasks 2-5, **save the code clips to a local file on your computer** (any text editor). I tend to favour any ascii text (notepad, textedit) editor because you really don't need any fancy capabilities of Word for this.


## Section 2.3: Load audio files from your computer / google drive (optional)
It is also possible to load any audio you have on your hard drive to the workspace of the notebook by clicking the folder icon (on the left) and choosing the file, waiting for the file to upload, and then referring to the file. These audio files will **NOT be saved** across the sessions unless you connect your google drive with Colab, which is easy (see [External data: Local Files, Drive, Sheets and Cloud Storage](https://colab.research.google.com/notebooks/io.ipynb)).   

In [ ]:
# NOTE! I have now commented these three lines out. If you want to use this idea, uncomment (remove the # signs from lines 2-4)
#x, sr = librosa.load('my_own_recording.wav') # if you want to start the segment from a specific point, add offset = [2 for example] in the command
#plt.figure(figsize=(12, 4))
#ipd.display(ipd.Audio(data = x, rate = sr))

# Section 3: Explore acoustic features across time

---
### _Learning Task 2_
---

Can you determine which of the four instrument sounds (clarinet, trumpet, harp and harpsichord) in the example above (Section 2.2) is the most bright and the least bright? Does this correspond with your own evaluation of how dark or bright (nasal) the instrument sounds?

To do this, you need to calculate _spectral centroid_ (a well-known acoustic correlate of brighness) of the signal, which you can simply get from librosa by invoking one of the pre-defined features: `feat = librosa.feature.spectral_centroid(signal)`. Just replace the `signal` with the input you had in mind, `harp`...

When you have done this, `feat` will contain a time series of the spectral centroid, and you can calculate the mean or the median of those values by `np.mean(feat)`. To display this summary, just can surround it with `print()` function (`print(np.mean(feat))`).

You can also the plot the spectral centroid over time by specifying: 


```
times = librosa.times_like(feat)
plt.figure(figsize=(12, 4))
plt.plot(times, feat[0])
plt.show()
```


In [ ]:
#write your analysis version here
feat = ...
print(...

# plot time-series
times = ...

The correct ranking of four instrument sounds in terms of the brightness is?

# Section 4: Analyse pitch
This is best done with a monophonic example, although `librosa` has algorithm to track all pitches as well. Here we extract the fundamental frequency of the trumpet solo using a probabilistic variant of the so-called YIN method. The variant is by Mauch and Dixon (2014) and the original technique was proposed by De Cheveigne and Kawahara (2002).

In [ ]:
filename = librosa.ex('trumpet')              # Mihai Sorohan - Monophonic trumpet recording
x, sr = librosa.load(filename)
ipd.display(ipd.Audio(data = x, rate = sr))   # code added to create playback object

# This is where the YIN algorithm is applied. fmin refers to the lower frequency threshold and fmax to higher frequence threshold.
f0, voiced_flag, voiced_probs = librosa.pyin(x, fmin = librosa.note_to_hz('C2'), fmax = librosa.note_to_hz('C6'))
times = librosa.times_like(f0)

# Plot the fundamental frequency, make the y-axis logarithmic for easier interpretation
plt.figure(figsize=(12, 4))
plt.plot(times, f0, 'ro:', linewidth = 2)
plt.xlabel('Time')
plt.ylabel('Hz')
ax=plt.gca()
ax.yaxis.set_major_formatter(librosa.display.LogHzFormatter())
ax.set(yticks=[320,320+80,320+80*2,320+80*3,320+80*4,320+80*5])
ax.set(ylim=[300, 800])
ax.yaxis.set_major_formatter(librosa.display.LogHzFormatter())

---
### _Learning Task 3_
---

Ok, the example above analysed a short trumpet melody. By let's turn to our four instrument sounds again. What is the pitch in Hz and in conventional note labelling that the four instrument sounds (clarinet, trumpet, harp, harpsichord) played?

You have the extraction routine define in the example above (`librosa.pyin...`)

And you can take the median of the f0 values by using this command that will ignore any missing values:

`Hz=np.nanmedian(f0)`

It is possible to convert Hz into note names by `librosa.hz_to_note(Hz))`

You can always look at the [librosa help pages](https://librosa.org/doc/latest/index.html)....


In [ ]:
f0, ... # extract F0
Hz .... # calculate median Hz from F0 time-series
        # Hz in note name?


# Section 5: Analyse spectrum

Spectrum is a useful representation of audio that shows the energy across frequencies. Try out plotting the spectrogram with the trumpet sound.

## Section 5.1: Spectrogram

In [ ]:
filename = librosa.ex('trumpet') #PROMPT: Try this out with the trumpet audio example. What is the spectrogram telling you?
x, sr = librosa.load(filename)  #

stft = np.abs(librosa.stft(x))
freqs = librosa.fft_frequencies(sr = sr)

ipd.display(ipd.Audio(data = x, rate = sr))
plt.figure(figsize=(12, 4))
librosa.display.specshow(librosa.amplitude_to_db(stft, ref = np.max), x_axis = 'time', y_axis = 'log')
plt.show()

## Section 5.2 Spectrum

Let's look at the spectrum of the instrument sounds.

Display the spectrum of one of the four example instruments.

In [ ]:
plt.figure(figsize=(12, 4))
stft = np.abs(librosa.stft(clarinet))
freqs = librosa.fft_frequencies(sr = sr)
Dmean = stft.mean(axis=1)/max(stft.mean(axis=1))
plt.plot(freqs, Dmean, color = 'm')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim([300, 1000])              # Modify at least this to see the peaks
#x=np.arange(Hz,Hz*10,Hz)          # Put labels to the multiples of the fundamental
#plt.xticks(x)
plt.show()

---
### _Learning Task 4_
---
_What does the spectrum of **clarinet** or **harpsichord** or **harp** look like? There is something special about odd and even harmonics and how the amplitudes of these qualities lead some instruments to sound woody or hollow. The first example is given for you but you need to adjust the range of frequencies shown in order to see multiple harmonics and you could also label the x axis based on the fundamental of the instrument sound._



# Section 6: Analyse onsets and tempo
This section estimates the strength of **possible onsets** and then detects the onsets that are stronger than a threshold defined in the algorithm. There is also a beat tracking that uses the onsets to estimate the plausible steady beats in the audio. This information is then used to infer tempo (in the next section).

## Section 6.1 Onsets and steady beat tracking
Look at the "Choice" example at first. Take the first 20 seconds (as duration) of this file. Afterwards, have a look at the 'brahms' example as well.

In [ ]:
filename = librosa.ex('choice')                                             # Try this example first
#filename = librosa.ex('brahms')                                            # Hungarian Dance no. 5
#filename = librosa.ex('nutcracker')                                        # Nutcracker
x, sr = librosa.load(filename, duration = 10)

o_env = librosa.onset.onset_strength(x, sr = sr,aggregate = np.median)      # Strength of onsets
times = librosa.times_like(o_env, sr = sr)                                  # Create timeline
onset_frames = librosa.onset.onset_detect(onset_envelope = o_env, sr = sr)  # Detect onsets

plt.figure(figsize=(12, 4))
librosa.display.waveplot(y = x,sr = sr, color = 'green')
plt.xlim([0,8])

plt.figure(figsize=(12, 4))
plt.plot(times, o_env, label = 'Onset strength')
plt.vlines(times[onset_frames], 0, o_env.max(), color = 'r', alpha = 0.9,
           linestyle='--', label='Onsets')
plt.legend()

# Sonify the detected beat events
tempo, beats = librosa.beat.beat_track(y=x, sr=sr,trim=True,onset_envelope=o_env,tightness=150)
y_beats = librosa.clicks(frames=beats, sr=sr)

combined = (x[0:len(y_beats)]+y_beats)/2
librosa.display.waveplot(y=abs(y_beats)*max(o_env),sr=sr,color='y')
plt.ylim([0,20])
plt.xlim([0,8])
plt.legend()
plt.show()
ipd.display(ipd.Audio(data=combined, rate=sr))

---
### _Learning Task 5_
---
_What are we seeing here? Can you explain the difference between onset strength (blue), detected onsets (red dotted lines) and beats (yellow)? Explore how these features behave in other examples (TIP: beginning of the code has the other examples commented out). How well are the onset detected and is the beat structure always what you think it should be?_

When you try another piece of music, you may have to alter the "tightness" parameters in the sonify section of the code.


## Section 6.2 Analyse tempo

---
### _Learning Task 6_
---

_Find **tempo** of the file that you analysed above. The tempo is indicated in Beats Per Minute (BPM).
*Tip. There is a command called* `beat.tempo` *that can be used to calculate the tempo.* The following script will not work until you have completed the function in the first line. Try this function for various example shown in the previous example and consider they the estimated tempo is plausible for each case. If not, why is that?_

In [ ]:
tempo = librosa.b...(x, aggregate = np.median)
print(np.round(tempo,1),'BPM')